# Imports & Setup
- We have to use multi-dimensional image viewers for TIF files — standard image viewing in skimage and cv2 only work for RGB images
- We can use Fiji (from the ImageJ suite), but napari allows us to perform similar viewing functionality via Python.

In [1]:
# Jupyter magic commands
%matplotlib inline
%reload_ext autoreload
%load_ext autoreload
%autoreload 2

# napari GUI enabled
%matplotlib qt 

# %gui qt           
# if having issues, maybe instead use QT console for jupyter? https://qtconsole.readthedocs.io/en/stable/

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [46]:
# general 
import numpy as np
import pandas as pd
import scipy.io
import pickle
import math
import os
import json

# image processing
import cv2
import PIL
from PIL import Image
import skimage
from skimage import exposure, filters, feature, transform
from skimage.color import rgb2gray
from skimage.io import imread
from dask import delayed # lazy load for large pipelines
from skimage.filters.rank import minimum as min_filter
from skimage.filters.rank import median as median_filter
from skimage.morphology import disk

# plotting
import seaborn as sns
import napari
import matplotlib.pyplot as plt

In [3]:
# serialization functions to store results
def serialize(obj, path):
    with open(path, "wb") as fh:
        pickle.dump(obj, fh)

def deserialize(path):
    with open(pathh, "rb") as fh:
        return pickle.load(fh)

# Viewing TIF metadata

In [21]:
metadata_path = 'data/channelNames.txt'
with open(metadata_path, mode='r') as metadata_txt:
    metadata = metadata_txt.readlines()   

In [30]:
channels = [i.strip("\n") for i in metadata]
print(channels)
print("\n ===> We have", len(channels), "channels!")

['HOECHST1', 'Vimentin', 'CD19', 'CD31', 'HOECHST2', 'CD85j', 'HLA-DR', 'Ki67', 'HOECHST3', 'CD15', 'CD274', 'FAP', 'HOECHST4', 'CD21', 'CD3', 'CD152', 'HOECHST5', 'CD4', 'CD69', 'CD2', 'HOECHST6', 'CD66', 'CD57', 'podoplanin', 'HOECHST7', 'CD8', 'HIF-1a', 'collagen IV', 'HOECHST8', 'CD11c', 'CD154', 'CD56', 'HOECHST9', 'CD54', 'IgM', 'MMP12', 'HOECHST10', 'CD7', 'CD36', 'CD9', 'HOECHST11', 'TCRyd', 'CD1c', 'CD49f', 'HOECHST12', 'CD38', 'CD45', 'CD16', 'HOECHST13', 'CD25', 'Foxp3', 'CD123', 'HOECHST14', 'CD40', 'cytokeratin', 'HLA-ABC', 'HOECHST15', 'CD45RA', 'CD90', 'CD127', 'HOECHST16', 'blank', 'CD278', 'CD185 (CXCR5)', 'HOECHST17', 'blank', 'CD117', 'Glucose Transporter', 'HOECHST18', 'blank', 'CD5', 'CD279', 'HOECHST19', 'blank', 'blank', 'CD273', 'HOECHST20', 'blank', 'blank', 'blank', 'HOECHST21', 'blank', 'blank', 'CD39-bio']

 ===> We have 84 channels!


# All dataset paths

- HN U54
- lung
- mouse
- sylvia's collab

# Link metadata to TIF files
- create a dictionary (called 'slices') that associates each channel name to its data

In [4]:
data_path = 'data/primary/'
dir_contents = os.listdir(data_path)
print(dir_contents)

['.DS_Store', 'reg005_montage.tif']


In [50]:
reg005_path = data_path + 'reg005_montage.tif'
reg005 = Image.open(reg005_path)
print()
# reg005 = np.array(reg005)

In [51]:
print("old shape:", reg005.shape)
D1,H,W,D2 = reg005.shape

reg005r = np.reshape(reg005, (D1*D2, H, W)) # reshape
print("new shape:", reg005r.shape)

AttributeError: 'TiffImageFile' object has no attribute 'shape'

In [31]:
data = [reg005r]

In [33]:
d = data[0]
slices = {}

for idx, channel in enumerate(channels):
    slices[channel] = d[idx,:,:]
#     print(channel)

# Filering out noise
- emply a median filter

In [41]:
f_size = 15 # filter size
dfil = disk(f_size)
dfil.shape
# print(dfil)

(31, 31)

In [42]:
# Median filter
for idx, channel in enumerate(channels):
    chi = slices[channel] # channel 0
    chif = median_filter(chi, dfil) # filtered
#     plt.imshow(chif)

/usr/local/lib/python3.7/site-packages/skimage/filters/rank/generic.py:119: UserWarning: Bad rank filter performance is expected due to a large number of bins (52407), equivalent to an approximate bitdepth of 15.7.
  out_dtype)
/usr/local/lib/python3.7/site-packages/skimage/filters/rank/generic.py:119: UserWarning: Bad rank filter performance is expected due to a large number of bins (51047), equivalent to an approximate bitdepth of 15.6.
  out_dtype)
/usr/local/lib/python3.7/site-packages/skimage/filters/rank/generic.py:119: UserWarning: Bad rank filter performance is expected due to a large number of bins (32340), equivalent to an approximate bitdepth of 15.0.
  out_dtype)
/usr/local/lib/python3.7/site-packages/skimage/filters/rank/generic.py:119: UserWarning: Bad rank filter performance is expected due to a large number of bins (55229), equivalent to an approximate bitdepth of 15.8.
  out_dtype)


KeyboardInterrupt: 

# Thresholding

# Viewing TIF files
- calling napari.view_image pops up a GUI for interactive viewing

In [ ]:
# viewer = napari.view_image(reg005r, rgb=False)

# Initial analysis

In [ ]:
# check mutual information of channels